In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import librosa
from scipy.stats import kurtosis
from scipy.stats import skew

def extract_features(y,sr=22050,n_fft=1024,hop_length=512):
    features = {'centroid': librosa.feature.spectral_centroid(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel(),
                'flux': librosa.onset.onset_strength(y=y, sr=sr).ravel(),
                'rmse': librosa.feature.rms(y, frame_length=n_fft, hop_length=hop_length).ravel(),
                'zcr': librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel(),
                'contrast': librosa.feature.spectral_contrast(y, sr=sr).ravel(),
                'bandwidth': librosa.feature.spectral_bandwidth(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel(),
                'flatness': librosa.feature.spectral_flatness(y, n_fft=n_fft, hop_length=hop_length).ravel(),
                'rolloff': librosa.feature.spectral_rolloff(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()}

    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, n_fft=n_fft, hop_length=hop_length, n_mfcc=20)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()

    # Get statistics from the vectors
    def get_feature_stats(features):
        result = {}
        for k, v in features.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)
        return result

    dict_agg_features = get_feature_stats(features)
    dict_agg_features['tempo'] = librosa.beat.tempo(y=y,sr=sr,hop_length=hop_length)[0]

    return dict_agg_features

from tkinter import filedialog
filename = filedialog.askopenfilename(title='open')
arr_features=[]
y, sr = librosa.load(filename, duration=30)
dict_features=extract_features(y=y,sr=sr)
arr_features.append(dict_features)
df=pd.DataFrame(data=arr_features)
print(df.head())
print(df.shape)
df.to_csv('test_data1.csv',index=False)

df=pd.read_csv('test_data1.csv')

X=df#df.drop('label',axis=1).values

df=pd.read_csv('train_data.csv')
df_t=pd.read_csv('train_data.csv')




X_train=df.drop('label',axis=1).values
y_train=df['label'].values

X_test=df_t.drop('label',axis=1).values
y_test=df_t['label'].values



import pandas as pd
from sklearn.ensemble import RandomForestClassifier

clfRF = RandomForestClassifier(max_depth=2, random_state=0)
clfRF.fit(X_train, y_train)
preds = clfRF.predict(X)

genres = ['blues' ,'classical',' country', 'disco', 'hiphop' ,'jazz',' metal',' pop',' reggae rock']
print('Predicted genres is ='+genres [preds[0]])


   centroid_max  centroid_min  centroid_mean  centroid_std  centroid_kurtosis  \
0   3137.991499    170.716008    1209.626134    531.229988          -0.300309   

   centroid_skew   flux_max  flux_min  flux_mean  flux_std  ...  mfcc_18_std  \
0       0.116321  14.315345       0.0   1.474048  1.744671  ...    15.098674   

   mfcc_18_kurtosis  mfcc_18_skew  mfcc_19_max  mfcc_19_min  mfcc_19_mean  \
0          0.478369      0.166576    54.110535   -47.103905     -1.168798   

   mfcc_19_std  mfcc_19_kurtosis  mfcc_19_skew       tempo  
0    16.093399         -0.161509       0.16078  107.666016  

[1 rows x 169 columns]
(1, 169)
Predicted genres is =jazz
